In [3]:
import getpass
from multiprocessing import process
import os
from dotenv import load_dotenv

load_dotenv()
os.getenv('ATLAS_VECTOR_SEARCH_INDEX_NAME')

'vector_index'

In [4]:
import getpass
import os

if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Enter API key for Cohere: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("command-r-plus", model_provider="cohere")

In [5]:
import getpass
import os

if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Enter API key for Cohere: ")

from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(model="embed-english-v3.0")

In [6]:
from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch

client = MongoClient(os.environ.get("MONGODB_CONNECTION_STRING"))

db = client["sample_mflix"]
collection = db["vector_embeddings"]
databases = client.list_database_names()
collection

Collection(Database(MongoClient(host=['cluster0-shard-00-01.tq7i2.mongodb.net:27017', 'cluster0-shard-00-02.tq7i2.mongodb.net:27017', 'cluster0-shard-00-00.tq7i2.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster0', authsource='admin', replicaset='atlas-648kkl-shard-0', tls=True), 'sample_mflix'), 'vector_embeddings')

In [7]:
from langchain_mongodb import MongoDBAtlasVectorSearch

vector_store = MongoDBAtlasVectorSearch.from_documents(
    embedding=embeddings,
    collection=collection,
    index_name=os.environ.get("ATLAS_VECTOR_SEARCH_INDEX_NAME", ""),
    relevance_score_fn="cosine",
)

TypeError: VectorStore.from_documents() missing 1 required positional argument: 'documents'

In [8]:
import bs4
from langchain import hub
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts import PromptTemplate


# Load and chunk contents of the blog
loader = PyPDFLoader("../test.pdf")
docs = loader.load()
# print(docs[0].page_content)
vector_store = MongoDBAtlasVectorSearch.from_documents(
    documents=docs,
    embedding=embeddings,
    collection=collection,
    index_name=os.environ.get("ATLAS_VECTOR_SEARCH_INDEX_NAME", ""),
    relevance_score_fn="cosine",
)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that 'I'm sorry, I don't have that information at the moment. Let me check and get back to you with the details.'. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""
)

print(vector_store.similarity_search("What is the name of the ml paper?"))


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    print(docs_content)
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

[Document(metadata={'_id': '67dfac96d3d6e340aaa700c3', 'producer': 'Acrobat Distiller 5.0.5 for Macintosh', 'creator': 'PyPDF', 'creationdate': '2004-05-07T10:39:44+05:30', 'moddate': '2004-09-28T08:44:01+08:00', 'title': 'Efficient Graph-Based Image Segmentation', 'source': '../test.pdf', 'total_pages': 15, 'page': 13, 'page_label': '14'}, page_content='under contract DAAL01-97-K-0104, and in part by\nNSF Research Infrastructure award CDA-9703470.\nWewould like to thank Shree Nayar, Jianbo Shi and\nDaphna Weinshall for use of their images. We would\nalso like to thank Jon Kleinberg, Eva Tardos and Dan\nRamras for discussions about the algorithm and the NP\nhardness result.\nReferences\nArya, S. and Mount, D.M. 1993. Approximate nearest neighbor\nsearching. In Proc. 4th Annual ACM-SIAM Symposium on Dis-\ncrete Algorithms, pp. 271–280.\nAusiello, G., Crescenzi, P., Gambosi, G., Kann, V ., Marchetti\nSpaccamela, A., and Protasi, M. (to appear). Complexity and Ap-\nproximation. Combinator

In [9]:
response = graph.invoke({"question": "who wrote the ML paper?"})
print(response)

174 Felzenszwalb and Huttenlocher
two components A and B and e2 is between one of
these components, say B, and some other component
C.
Now we show that e1 causes a merge when con-
sidered after e2 exactly when it would cause a merge
if considered before e2. First, suppose that e1 causes
a merge when considered before e2. This implies
w(e1) ≤ MInt(A, B). If e2 were instead considered
before e1, either e2 would not cause a merge and triv-
ially e1 would still cause a merge, or e2 would cause
a merge in which case the new component B ∪ C
would have Int(B ∪ C) = w(e2) = w(e1). So we
know w(e1) ≤ MInt(A, B ∪ C) which implies e1 will
still cause a merge. On the other hand, suppose that
e1 does not cause a merge when considered before
e2. This implies w(e1) > MInt(A, B). Then either
w(e1) > Int(A) + τ(A), in which case this would
still be true if e2 were considered ﬁrst (because e2
does not touch A), or w(e1) > Int(B) + τ(B). In this
second case, if e2 were considered ﬁrst it could not
cause 

In [7]:
from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [ ]:
from langgraph.graph import MessagesState

graph_builder = StateGraph(MessagesState)

In [9]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [10]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [11]:
input_message = "What is the algorithm of the paper used."

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the algorithm of the paper used.
================================== Ai Message ==================================

I will search for the paper used and then search for the algorithm used in that paper.
Tool Calls:
  retrieve (retrieve_m06ygj77r272)
 Call ID: retrieve_m06ygj77r272
  Args:
    query: paper used
================================= Tool Message =================================
Name: retrieve


================================== Ai Message ==================================

I don't know.
